# Project: Train a Quadcopter How to Fly

Design an agent to fly a quadcopter, and then train it using a reinforcement learning algorithm of your choice! 

Try to apply the techniques you have learnt, but also feel free to come up with innovative ideas and test them.

## Instructions

Take a look at the files in the directory to better understand the structure of the project. 

- `task.py`: Define your task (environment) in this file.
- `agents/`: Folder containing reinforcement learning agents.
    - `policy_search.py`: A sample agent has been provided here.
    - `agent.py`: Develop your agent here.
- `physics_sim.py`: This file contains the simulator for the quadcopter.  **DO NOT MODIFY THIS FILE**.

For this project, you will define your own task in `task.py`.  Although we have provided a example task to get you started, you are encouraged to change it.  Later in this notebook, you will learn more about how to amend this file.

You will also design a reinforcement learning agent in `agent.py` to complete your chosen task.  

You are welcome to create any additional files to help you to organize your code.  For instance, you may find it useful to define a `model.py` file defining any needed neural network architectures.

## Controlling the Quadcopter

We provide a sample agent in the code cell below to show you how to use the sim to control the quadcopter.  This agent is even simpler than the sample agent that you'll examine (in `agents/policy_search.py`) later in this notebook!

The agent controls the quadcopter by setting the revolutions per second on each of its four rotors.  The provided agent in the `Basic_Agent` class below always selects a random action for each of the four rotors.  These four speeds are returned by the `act` method as a list of four floating-point numbers.  

For this project, the agent that you will implement in `agents/agent.py` will have a far more intelligent method for selecting actions!

In [1]:
import random

class Basic_Agent():
    def __init__(self, task):
        self.task = task
    
    def act(self):
        new_thrust = random.gauss(450., 25.)
        return [new_thrust + random.gauss(0., 1.) for x in range(4)]

Run the code cell below to have the agent select actions to control the quadcopter.  

Feel free to change the provided values of `runtime`, `init_pose`, `init_velocities`, and `init_angle_velocities` below to change the starting conditions of the quadcopter.

The `labels` list below annotates statistics that are saved while running the simulation.  All of this information is saved in a text file `data.txt` and stored in the dictionary `results`.  

In [2]:
%load_ext autoreload
%autoreload 2

import csv
import numpy as np
from task import Task

# Modify the values below to give the quadcopter a different starting position.
runtime = 5.                                     # time limit of the episode
init_pose = np.array([0., 0., 10., 0., 0., 0.])  # initial pose
init_velocities = np.array([0., 0., 0.])         # initial velocities
init_angle_velocities = np.array([0., 0., 0.])   # initial angle velocities
file_output = 'data.txt'                         # file name for saved results

# Setup
task = Task(init_pose, init_velocities, init_angle_velocities, runtime)
agent = Basic_Agent(task)
done = False
labels = ['time', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'x_velocity',
          'y_velocity', 'z_velocity', 'phi_velocity', 'theta_velocity',
          'psi_velocity', 'rotor_speed1', 'rotor_speed2', 'rotor_speed3', 'rotor_speed4']
results = {x : [] for x in labels}

# Run the simulation, and save the results.
with open(file_output, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(labels)
    while True:
        rotor_speeds = agent.act()
        _, _, done = task.step(rotor_speeds)
        to_write = [task.sim.time] + list(task.sim.pose) + list(task.sim.v) + list(task.sim.angular_v) + list(rotor_speeds)
        for ii in range(len(labels)):
            results[labels[ii]].append(to_write[ii])
        writer.writerow(to_write)
        if done:
            break

ImportError: cannot import name 'Task'

Run the code cell below to visualize how the position of the quadcopter evolved during the simulation.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['x'], label='x')
plt.plot(results['time'], results['y'], label='y')
plt.plot(results['time'], results['z'], label='z')
plt.legend()
_ = plt.ylim()

The next code cell visualizes the velocity of the quadcopter.

In [ ]:
plt.plot(results['time'], results['x_velocity'], label='x_hat')
plt.plot(results['time'], results['y_velocity'], label='y_hat')
plt.plot(results['time'], results['z_velocity'], label='z_hat')
plt.legend()
_ = plt.ylim()

Next, you can plot the Euler angles (the rotation of the quadcopter over the $x$-, $y$-, and $z$-axes),

In [ ]:
plt.plot(results['time'], results['phi'], label='phi')
plt.plot(results['time'], results['theta'], label='theta')
plt.plot(results['time'], results['psi'], label='psi')
plt.legend()
_ = plt.ylim()

before plotting the velocities (in radians per second) corresponding to each of the Euler angles.

In [ ]:
plt.plot(results['time'], results['phi_velocity'], label='phi_velocity')
plt.plot(results['time'], results['theta_velocity'], label='theta_velocity')
plt.plot(results['time'], results['psi_velocity'], label='psi_velocity')
plt.legend()
_ = plt.ylim()

Finally, you can use the code cell below to print the agent's choice of actions.  

In [ ]:
plt.plot(results['time'], results['rotor_speed1'], label='Rotor 1 revolutions / second')
plt.plot(results['time'], results['rotor_speed2'], label='Rotor 2 revolutions / second')
plt.plot(results['time'], results['rotor_speed3'], label='Rotor 3 revolutions / second')
plt.plot(results['time'], results['rotor_speed4'], label='Rotor 4 revolutions / second')
plt.legend()
_ = plt.ylim()

When specifying a task, you will derive the environment state from the simulator.  Run the code cell below to print the values of the following variables at the end of the simulation:
- `task.sim.pose` (the position of the quadcopter in ($x,y,z$) dimensions and the Euler angles),
- `task.sim.v` (the velocity of the quadcopter in ($x,y,z$) dimensions), and
- `task.sim.angular_v` (radians/second for each of the three Euler angles).

In [ ]:
# the pose, velocity, and angular velocity of the quadcopter at the end of the episode
print(task.sim.pose)
print(task.sim.v)
print(task.sim.angular_v)

In the sample task in `task.py`, we use the 6-dimensional pose of the quadcopter to construct the state of the environment at each timestep.  However, when amending the task for your purposes, you are welcome to expand the size of the state vector by including the velocity information.  You can use any combination of the pose, velocity, and angular velocity - feel free to tinker here, and construct the state to suit your task.

## The Task

A sample task has been provided for you in `task.py`.  Open this file in a new window now. 

The `__init__()` method is used to initialize several variables that are needed to specify the task.  
- The simulator is initialized as an instance of the `PhysicsSim` class (from `physics_sim.py`).  
- Inspired by the methodology in the original DDPG paper, we make use of action repeats.  For each timestep of the agent, we step the simulation `action_repeats` timesteps.  If you are not familiar with action repeats, please read the **Results** section in [the DDPG paper](https://arxiv.org/abs/1509.02971).
- We set the number of elements in the state vector.  For the sample task, we only work with the 6-dimensional pose information.  To set the size of the state (`state_size`), we must take action repeats into account.  
- The environment will always have a 4-dimensional action space, with one entry for each rotor (`action_size=4`). You can set the minimum (`action_low`) and maximum (`action_high`) values of each entry here.
- The sample task in this provided file is for the agent to reach a target position.  We specify that target position as a variable.

The `reset()` method resets the simulator.  The agent should call this method every time the episode ends.  You can see an example of this in the code cell below.

The `step()` method is perhaps the most important.  It accepts the agent's choice of action `rotor_speeds`, which is used to prepare the next state to pass on to the agent.  Then, the reward is computed from `get_reward()`.  The episode is considered done if the time limit has been exceeded, or the quadcopter has travelled outside of the bounds of the simulation.

In the next section, you will learn how to test the performance of an agent on this task.

## The Agent

The sample agent given in `agents/policy_search.py` uses a very simplistic linear policy to directly compute the action vector as a dot product of the state vector and a matrix of weights. Then, it randomly perturbs the parameters by adding some Gaussian noise, to produce a different policy. Based on the average reward obtained in each episode (`score`), it keeps track of the best set of parameters found so far, how the score is changing, and accordingly tweaks a scaling factor to widen or tighten the noise.

Run the code cell below to see how the agent performs on the sample task.

In [ ]:
import sys
import pandas as pd
from agents.policy_search import PolicySearch_Agent
from task import Task

num_episodes = 1000
target_pos = np.array([0., 0., 10.])
task = Task(target_pos=target_pos)
agent = PolicySearch_Agent(task) 

for i_episode in range(1, num_episodes+1):
    state = agent.reset_episode() # start a new episode
    while True:
        action = agent.act(state) 
        next_state, reward, done = task.step(action)
        agent.step(reward, done)
        state = next_state
        if done:
            print("\rEpisode = {:4d}, score = {:7.3f} (best = {:7.3f}), noise_scale = {}".format(
                i_episode, agent.score, agent.best_score, agent.noise_scale), end="")  # [debug]
            break
    sys.stdout.flush()

This agent should perform very poorly on this task.  And that's where you come in!

## Define the Task, Design the Agent, and Train Your Agent!

Amend `task.py` to specify a task of your choosing.  If you're unsure what kind of task to specify, you may like to teach your quadcopter to takeoff, hover in place, land softly, or reach a target pose.  

After specifying your task, use the sample agent in `agents/policy_search.py` as a template to define your own agent in `agents/agent.py`.  You can borrow whatever you need from the sample agent, including ideas on how you might modularize your code (using helper methods like `act()`, `learn()`, `reset_episode()`, etc.).

Note that it is **highly unlikely** that the first agent and task that you specify will learn well.  You will likely have to tweak various hyperparameters and the reward function for your task until you arrive at reasonably good behavior.

As you develop your agent, it's important to keep an eye on how it's performing. Use the code above as inspiration to build in a mechanism to log/save the total rewards obtained in each episode to file.  If the episode rewards are gradually increasing, this is an indication that your agent is learning.

#### This code was run in 2 separate files. One for take off, one for landing, here is some sample code for review:
```python
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from agents.agent import ValueNetwork, PolicyNetwork, OUNoise, ReplayBuffer, ddpg_update
from task import LandTask, TakeOffTask

import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

state_dim = 18
action_dim = 4
hidden_dim = 256
action_space = {'low': 0, 'high': 900, 'action': 4}
ou_noise = OUNoise()

value_net = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

target_value_net = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
target_policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_lr = 1e-3
policy_lr = 1e-4

value_optimizer = optim.Adam(value_net.parameters(), lr=value_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)

value_criterion = nn.MSELoss()

replay_buffer_size = 1000000
replay_buffer = ReplayBuffer(replay_buffer_size)

episodes = 10001
steps = 500
episode = 0
total_rewards = []
total_steps = []
batch_size = 128
logs = []
#env = LandTask()
env = TakeOffTask()

while episode < episodes:
    print('starting run {}'.format(episode))
    state = env.reset()
    ou_noise.reset()
    episode_reward = 0
    episode += 1
    local_log = []

    for step in range(steps):
        action = policy_net.get_action(state)
        action = ou_noise.get_action(action, step)
        next_state, reward, done = env.step(action)

        replay_buffer.push(state, action, reward, next_state, done)
        if len(replay_buffer) > batch_size:
            ddpg_update(batch_size,replay_buffer, value_net, policy_net, target_policy_net, target_value_net, value_criterion, policy_optimizer, value_optimizer)

        state = next_state
        episode_reward += reward
        local_log.append(reward)

        if done:
            total_rewards.append(episode_reward)
            total_steps.append(step)
            logs.append(local_log)
            break
    else:
        print('default {}'.format(episode_reward))
        total_rewards.append(episode_reward)
        total_steps.append(step)
        logs.append(local_log)

np.savetxt("takeoff_rewards.csv", logs, delimiter=",", fmt='%s')

plt.plot(total_rewards)
plt.savefig('takeoff_rewards.png')
plt.plot('total_steps')
plt.savefig('takeoff_steps.png')

```


## Plot the Rewards

Once you are satisfied with your performance, plot the episode rewards, either from a single run, or averaged over multiple runs. 

In [4]:
import pandas as pd

## Landing Rewards

<img src='landing_rewards.png'>

In [7]:
landing_rewards = pd.read_csv('landing_rewards.csv', header=None)

landing_rewards.head(50)

,0,1,2,3,4,5,6,7,8,9,10
0,-1.787352,6.041954,6.068716,6.093521,6.115261,6.132464,6.143149,6.142424,6.117816,6.071078,6.010752
1,1.062865,6.041954,6.068716,6.093521,6.115261,6.132464,6.143149,6.142423,6.117815,6.071077,6.010751
2,1.062869,6.041954,6.068717,6.093522,6.115262,6.132466,6.143152,6.142427,6.117819,6.071082,6.010755
3,1.062844,6.041954,6.068716,6.093521,6.115261,6.132464,6.143149,6.142424,6.117816,6.071079,6.010754
4,1.062862,6.041954,6.068717,6.093521,6.115262,6.132466,6.143151,6.142425,6.117817,6.071080,6.010753
5,1.062854,6.041954,6.068716,6.093521,6.115261,6.132465,6.143150,6.142424,6.117816,6.071080,6.010754
6,1.062859,6.041954,6.068716,6.093521,6.115261,6.132464,6.143149,6.142423,6.117815,6.071078,6.010751
7,1.062866,6.041954,6.068716,6.093521,6.115261,6.132465,6.143150,6.142424,6.117816,6.071078,6.010752
8,1.062862,6.041954,6.068716,6.093521,6.115261,6.132464,6.143149,6.142423,6.117814,6.071077,6.010751
9,1.062870,6.041954,6.068716,6.093521,6.115261,6.132465,6.143149,6.142424,6.117815,6.071078,6.010751


## Takeoff Rewards

<img src='takeoff_rewards.png'>

In [8]:
takeoff_rewards = pd.read_csv('takeoff_rewards.csv', header=None)

takeoff_rewards.head(50)

,0
0,-20.357295
1,0.773962
2,-8.773962
3,0.773962
4,-11.305202
5,3.305202
6,-8.773962
7,-4.531240
8,3.305202
9,-8.773962


## Reflections

**Question 1**: Describe the task that you specified in `task.py`.  How did you design the reward function?

**Answer**:
I created 2 tasks for this project. One is a take off task and the other is a landing task:

<b>Take Off:</b>
This is supposed to be the simple one. The agent starts in a 0,0,0,0,0,0 position and has to fly straight up. I assigned rewards based on position, distance and error loss. I used huber loss to compare the 6 value vector (3 for location and 3 for angles). I also used the euclidian distance between the current position and the target. The agent would get a point if it lowered the distance and also got a point for lowering the loss. It would lose a point if it got further away or created more loss.

<b>Landing Task:</b>
The landing task scoring was done the same way except that I started the agent in a random position and it would have to land straight and on target. 

**Question 2**: Discuss your agent briefly, using the following questions as a guide:

- What learning algorithm(s) did you try? What worked best for you?

I followed the instructions and created an actor/critic model as this seemed the best for continuous networks. I really tried to get a normal DQN to work but just could not figure out a way to map the actions in a continous space. As suggested in the instrcutions, I used experience replay to speed up the training as well as minimize temporal correlation. Noise was also added to help the agent explore.  I ended up using pytorch for all this just to learn more about pytorch.

- What was your final choice of hyperparameters (such as $\alpha$, $\gamma$, $\epsilon$, etc.)?

The actor learning rate was set to 1-e4, the critic learning rate was set to 1-e3. The replay buffer size was set to 1000000 with a batch size of 128.


- What neural network architecture did you use (if any)? Specify layers, sizes, activation functions, etc.

I used a simple 3 layer NN for the actor and the critic with a hidden dimension size of 256. The critic used relu activators while the actor used relu and finished with tanh. I used both MSE and SmoothL1loss (Huber Loss) as loss functions.

**Question 3**: Using the episode rewards plot, discuss how the agent learned over time.

- Was it an easy task to learn or hard?

It was definetly not easy. Based on the graph, the agent seemed to learn very quickly though.

- Was there a gradual learning curve, or an aha moment?

There was an aha moment after trying to get this agent to learn using a DQN. Much of the time was spent just trying to understand the physics sim and create an environment for the agent to interact with. All the examples you see online use Gym so this project let me dig into the internals. For sure the most challenging project of this course.

- How good was the final performance of the agent? (e.g. mean rewards over the last 10 episodes)

Based on the data, the agent seemed to learn very fast and held it at the cap rewards. I am skeptical on the learning speed though.

**Question 4**: Briefly summarize your experience working on this project. You can use the following prompts for ideas.

- What was the hardest part of the project? (e.g. getting started, plotting, specifying the task, etc.)

The hardest part was just just starting it. Having to create the environment and dig into what the physics sim file was actually doing took up a fair amount of time.

- Did you find anything interesting in how the quadcopter or your agent behaved?

I noticed a huge spike in the training. I think the take off task is too easy maybe? The landing task rewards would just sky rocket up and stay pretty much stationary.



## References used
- https://www.udemy.com/reinforcement-learning-with-pytorch/ (I took another course to complete this project :) )
- https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
- https://github.com/vy007vikas/PyTorch-ActorCriticRL
- https://github.com/rgilman33/simple-A2C
- https://www.packtpub.com/big-data-and-business-intelligence/deep-learning-pytorch
